In [9]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [10]:
LOCAL = False

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [11]:
%%capture pwd
!pwd

In [12]:
# uid = "2021-05-05-which-firms-and-industries-have-been-most-affected-by-covid-update"  # article unique ID
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
height, width

(300, 500)

# Fig 1

In [15]:
df = pd.DataFrame(
    [
        {"employees": "All businesses", "year": 2018, "value": 14.0},
        {"employees": "All businesses", "year": 2019, "value": 11.6},
        {"employees": "1000 or more employees", "year": 2018, "value": 56.3},
        {"employees": "1000 or more employees", "year": 2019, "value": 59.2},
        {"employees": "250 to 999 employees", "year": 2018, "value": 45.3},
        {"employees": "250 to 999 employees", "year": 2019, "value": 42.1},
        {"employees": "50 to 249 employees", "year": 2018, "value": 35.5},
        {"employees": "50 to 249 employees", "year": 2019, "value": 32.8},
        {"employees": "10 to 49 employees", "year": 2018, "value": 23.2},
        {"employees": "10 to 49 employees", "year": 2019, "value": 27.2},
        {"employees": "0 to 9 employees", "year": 2018, "value": 12.7},
        {"employees": "0 to 9 employees", "year": 2019, "value": 9.8},
    ]
)

In [463]:
f = "fig1_employees"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
df.head()

,employees,year,value
0,All businesses,2018,14.0
1,All businesses,2019,11.6
2,1000 or more employees,2018,56.3
3,1000 or more employees,2019,59.2
4,250 to 999 employees,2018,45.3


In [ ]:
base = alt.Chart(f1).encode(
    x=alt.X(
        "value:Q",
        title="",
        axis=alt.Axis(
            grid=False,
            title="%",
            titleAnchor="end",
            titleY=-15,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
    ),
    y=alt.Y("employees:N", axis=None, sort=[]),
)
points1 = base.mark_point(
    size=40, color=colors["eco-turquiose"], fill=colors["eco-turquiose"], opacity=0.8
).transform_filter("datum.year==2018")
points2 = base.mark_point(
    size=40, color=colors["eco-blue"], fill=colors["eco-blue"], opacity=0.8
).transform_filter("datum.year==2019")
value1 = points1.mark_text(
    color=colors["eco-turquiose"], size=10, align="center", yOffset=12
).encode(text="value:N")
value2 = points2.mark_text(
    color=colors["eco-blue"], size=10, align="center", yOffset=-11
).encode(text="value:N")
text2 = (
    points2.mark_text(color=colors["eco-gray"], size=10, align="right", xOffset=-10)
    .encode(text="employees:N")
    .transform_filter(
        alt.FieldOneOfPredicate(
            field="employees",
            oneOf=[
                "All businesses",
                "250 to 999 employees",
                "0 to 9 employees",
                "50 to 249 employees",
            ],
        )
    )
)
text1 = (
    points1.mark_text(color=colors["eco-gray"], size=10, align="right", xOffset=-10)
    .encode(text="employees:N")
    .transform_filter(
        alt.FieldOneOfPredicate(
            field="employees", oneOf=["10 to 49 employees", "1000 or more employees"]
        )
    )
)
lines = (
    base.mark_errorbar()
    .encode(
        x=alt.X("2018:Q", title="%"),
        x2=alt.X2("2019:Q"),
        y=alt.Y("employees:N", title=""),
        color=alt.condition(
            "datum['2018']>datum['2019']",
            alt.ColorValue(colors["eco-blue"]),
            alt.ColorValue(colors["eco-blue"]),
        ),
    )
    .transform_pivot("year", groupby=["employees"], value="value")
)
arrows2 = text2.mark_point(
    size=5,
    angle=30,
    xOffset=7,
    color=colors["eco-blue"],
    shape="triangle",
    fill=colors["eco-blue"],
    opacity=0.8,
).transform_filter("datum.year==2019")
arrows1 = (
    base.mark_point(
        size=5,
        angle=210,
        xOffset=-7,
        color=colors["eco-blue"],
        shape="triangle",
        fill=colors["eco-blue"],
        opacity=0.8,
    )
    .transform_filter("datum.year==2019")
    .transform_filter(
        alt.FieldOneOfPredicate(
            field="employees", oneOf=["10 to 49 employees", "1000 or more employees"]
        )
    )
)
label = (
    alt.Chart(
        pd.DataFrame(
            [
                {
                    "x": 51,
                    "y": "10 to 49 employees",
                    "t": "2018",
                    "c": colors["eco-turquiose"],
                },
                {
                    "x": 54.8,
                    "y": "10 to 49 employees",
                    "t": " ➡ 2019",
                    "c": colors["eco-blue"],
                },
            ]
        )
    )
    .mark_text(yOffset=-26, align="left", size=10, baseline="middle")
    .encode(text="t", x="x:Q", y="y:N", color=alt.Color("c:N", scale=None))
)
layer = (
    (
        lines
        + points1
        + points2
        + arrows1
        + arrows2
        + text1
        + text2
        + label
        + value1
        + value2
    )
    .configure_view(stroke=None)
    .properties(title="")
    .properties(height=200, width=400)
)
layer.save("visualisation/" + f + ".json")
layer

# Fig 2

In [ ]:
df = pd.DataFrame(
    [
        {"employees": "All firms", "year": 2018, "value": 11.6},
        {"employees": "All firms", "year": 2019, "value": 28.6},
        {"employees": "Retail", "year": 2018, "value": 36.1},
        {"employees": "Retail", "year": 2019, "value": 48.3},
        {"employees": "Other services", "year": 2018, "value": 6.7},
        {"employees": "Other services", "year": 2019, "value": 18.4},
        {"employees": "Information and communication", "year": 2018, "value": 5.4},
        {"employees": "Information and communication", "year": 2019, "value": 24.4},
        {"employees": "Wholesale", "year": 2018, "value": 19.1},
        {"employees": "Wholesale", "year": 2019, "value": 44.1},
        {"employees": "Construction", "year": 2018, "value": 4.1},
        {"employees": "Construction", "year": 2019, "value": 5.5},
        {"employees": "Manufacturing", "year": 2018, "value": 20.1},
        {"employees": "Manufacturing", "year": 2019, "value": 22.8},
    ]
)

In [ ]:
f = "fig2_sectors"
f2 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2 = df
df.head()

In [ ]:
base = alt.Chart(f2).encode(
    x=alt.X(
        "value:Q",
        title="",
        axis=alt.Axis(
            grid=False,
            title="%",
            titleAnchor="end",
            titleY=-15,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
    ),
    y=alt.Y("employees:N", axis=None, sort=[]),
)
points1 = base.mark_point(
    size=40, color=colors["eco-turquiose"], fill=colors["eco-turquiose"], opacity=0.8
).transform_filter("datum.year==2018")
points2 = base.mark_point(
    size=40, color=colors["eco-blue"], fill=colors["eco-blue"], opacity=0.8
).transform_filter("datum.year==2019")
value1 = points1.mark_text(
    color=colors["eco-turquiose"], size=10, align="right", xOffset=-8
).encode(text="value:N")
value2 = points2.mark_text(
    color=colors["eco-blue"], size=10, align="left", xOffset=8
).encode(text="value:N")
text1 = points1.mark_text(
    color=colors["eco-gray"],
    size=10,
    align="left",
    xOffset=8,
    yOffset=-11,
).encode(text="employees:N")
lines = (
    base.mark_errorbar()
    .encode(
        x=alt.X("2018:Q", title="%"),
        x2=alt.X2("2019:Q"),
        y=alt.Y("employees:N", title=""),
        color=alt.condition(
            "datum['2018']>datum['2019']",
            alt.ColorValue(colors["eco-blue"]),
            alt.ColorValue(colors["eco-blue"]),
        ),
    )
    .transform_pivot("year", groupby=["employees"], value="value")
)
label = (
    alt.Chart(
        pd.DataFrame(
            [
                {
                    "x": 54,
                    "y": "Construction",
                    "t": "Including micro entreprises",
                    "c": colors["eco-turquiose"],
                },
                {
                    "x": 54,
                    "y": "Information and communication",
                    "t": "Excluding micro entreprises",
                    "c": colors["eco-blue"],
                },
            ]
        )
    )
    .mark_text(align="right", size=10, baseline="middle")
    .encode(text="t", x="x:Q", y="y:N", color=alt.Color("c:N", scale=None))
)
layer = (
    (lines + points1 + points2 + text1 + value1 + value2 + label)
    .configure_view(stroke=None)
    .properties(title="")
    .properties(height=240, width=400)
)
layer.save("visualisation/" + f + ".json")
layer

# Fig 3

In [ ]:
df = pd.DataFrame(
    [
        {
            "employees": "1000 or more employees",
            "year": "Order tracking available online",
            "value": 31.2,
        },
        {
            "employees": "1000 or more employees",
            "year": "Online ordering or reservation/booking",
            "value": 49.4,
        },
        {
            "employees": "1000 or more employees",
            "year": "Website, own or third party",
            "value": 96.9,
        },
        {
            "employees": "250 to 999 employees",
            "year": "Order tracking available online",
            "value": 13.7,
        },
        {
            "employees": "250 to 999 employees",
            "year": "Online ordering or reservation/booking",
            "value": 32.8,
        },
        {
            "employees": "250 to 999 employees",
            "year": "Website, own or third party",
            "value": 95.4,
        },
        {
            "employees": "50 to 249 employees",
            "year": "Order tracking available online",
            "value": 7.1,
        },
        {
            "employees": "50 to 249 employees",
            "year": "Online ordering or reservation/booking",
            "value": 27.8,
        },
        {
            "employees": "50 to 249 employees",
            "year": "Website, own or third party",
            "value": 92.2,
        },
        {
            "employees": "10 to 49 employees",
            "year": "Order tracking available online",
            "value": 6.8,
        },
        {
            "employees": "10 to 49 employees",
            "year": "Online ordering or reservation/booking",
            "value": 29.4,
        },
        {
            "employees": "10 to 49 employees",
            "year": "Website, own or third party",
            "value": 81.4,
        },
        {
            "employees": "0 to 9 employees",
            "year": "Order tracking available online",
            "value": 6.1,
        },
        {
            "employees": "0 to 9 employees",
            "year": "Online ordering or reservation/booking",
            "value": 19.3,
        },
        {
            "employees": "0 to 9 employees",
            "year": "Website, own or third party",
            "value": 41.1,
        },
    ]
)

In [ ]:
f = "fig3_online"
f3 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f3.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f3 = df
df.head()

In [ ]:
bars = alt.Chart(f3).encode(
    x=alt.X(
        "value:Q",
        stack=False,
        title="",
        axis=alt.Axis(
            grid=False,
            title="%",
            titleAnchor="end",
            titleX=415,
            titleY=7,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
    ),
    y=alt.Y("year:N", title="", axis=None),
)

bars1 = bars.mark_bar(
    size=11, yOffset=-14, color=colors["eco-turquiose"], opacity=0.8
).transform_filter("datum.employees=='10 to 49 employees'")
bars2 = bars.mark_bar(
    size=11, yOffset=-26, color=colors["eco-green"], opacity=0.8
).transform_filter("datum.employees=='0 to 9 employees'")
bars3 = bars.mark_bar(
    size=11, yOffset=-2, color=colors["eco-light-blue"], opacity=0.8
).transform_filter("datum.employees=='50 to 249 employees'")
bars4 = bars.mark_bar(
    size=11, yOffset=10, color=colors["eco-mid-blue"], opacity=0.8
).transform_filter("datum.employees=='250 to 999 employees'")
bars5 = bars.mark_bar(
    size=11, yOffset=22, color=colors["eco-blue"], opacity=0.8
).transform_filter("datum.employees=='1000 or more employees'")
labels1 = bars1.mark_text(
    align="left",
    xOffset=5,
    yOffset=-14,
    color=colors["eco-turquiose"],
).encode(text="value:N")
labels2 = bars2.mark_text(
    align="left",
    xOffset=5,
    yOffset=-26,
    color=colors["eco-green"],
).encode(text="value:N")
labels3 = bars3.mark_text(
    align="left",
    xOffset=5,
    yOffset=-2,
    color=colors["eco-light-blue"],
).encode(text="value:N")
labels4 = bars4.mark_text(
    align="left",
    xOffset=5,
    yOffset=10,
    color=colors["eco-mid-blue"],
).encode(text="value:N")
labels5 = bars5.mark_text(
    align="left",
    xOffset=5,
    yOffset=22,
    color=colors["eco-blue"],
).encode(text="value:N")
titles = (
    bars.mark_text(
        align="left", size=11, baseline="bottom", yOffset=-35, color=colors["eco-gray"]
    )
    .encode(text="year:N", x="x0:Q", y="year:N")
    .transform_filter("datum.employees=='1000 or more employees'")
    .transform_calculate(x0="0")
)
legend1 = (
    bars1.mark_text(align="right", size=10, yOffset=-13, color=colors["eco-turquiose"])
    .encode(text="employees:N", x="x0:Q")
    .transform_filter("datum.year=='Online ordering or reservation/booking'")
    .transform_calculate(x0="100")
)
legend2 = (
    bars2.mark_text(align="right", size=10, yOffset=-25, color=colors["eco-green"])
    .encode(text="employees:N", x="x0:Q")
    .transform_filter("datum.year=='Online ordering or reservation/booking'")
    .transform_calculate(x0="100")
)
legend3 = (
    bars3.mark_text(align="right", size=10, yOffset=-1, color=colors["eco-light-blue"])
    .encode(text="employees:N", x="x0:Q")
    .transform_filter("datum.year=='Online ordering or reservation/booking'")
    .transform_calculate(x0="100")
)
legend4 = (
    bars4.mark_text(align="right", size=10, yOffset=11, color=colors["eco-mid-blue"])
    .encode(text="employees:N", x="x0:Q")
    .transform_filter("datum.year=='Online ordering or reservation/booking'")
    .transform_calculate(x0="100")
)
legend5 = (
    bars5.mark_text(align="right", size=10, yOffset=23, color=colors["eco-dark-blue"])
    .encode(text="employees:N", x="x0:Q")
    .transform_filter("datum.year=='Online ordering or reservation/booking'")
    .transform_calculate(x0="100")
)

layer = (
    (
        (bars1 + legend1)
        + (bars2 + legend2)
        + (bars3 + legend3)
        + (bars4 + legend4)
        + (bars5 + legend5)
        + labels1
        + labels2
        + labels3
        + labels4
        + labels5
        + titles
    )
    .configure_view(stroke=None)
    .properties(title="")
    .properties(height=300, width=400)
)
layer.save("visualisation/" + f + ".json")
layer